In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
# Importing Classifier Modules
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('C:/Users/UL494YB/OneDrive - EY/Desktop/Uchicago/Data Mining Principle/session 5/Assignment 3/Assignment_3/Loan_Prediction/Data for Cleaning & Modeling.csv')
df.head()

Data Cleaning/Prepping - Training Data

In [ ]:
#based on my analysis, I decided to use the following features for my training data
df_reduced=df[['X1','X4','X5','X7','X8','X11',
        'X13', 'X14','X17', 'X21',
       'X22','X24', 'X27', 'X28', 'X29', 'X30', 'X31',
       'X32']]
df_reduced.head(5)

In [ ]:
print(len(df_reduced['X1'].unique()))
df_reduced['X1'].value_counts()

In [ ]:
df_reduced['X1']=df_reduced['X1'].replace({'%':''}, regex = True)
df_reduced['X1']=df_reduced['X1'].astype(float)

In [ ]:
df_reduced['X1'].describe()
#getting summary stats on my target variable

In [ ]:
df_reduced['X4']=df_reduced['X4'].replace({'\$':''}, regex = True)
df_reduced['X4']=df_reduced['X4'].replace({',':''}, regex = True)
df_reduced['X4']=df_reduced['X4'].astype(float)

In [ ]:
df_reduced['X5']=df_reduced['X5'].replace({'\$':''}, regex = True)
df_reduced['X5']=df_reduced['X5'].replace({',':''}, regex = True)
df_reduced['X5']=df_reduced['X5'].astype(float)

In [ ]:
df_reduced['X7']=df_reduced['X7'].str.slice(1,3)
df_reduced['X7'].value_counts()

In [ ]:
d2 = {'36': 1, '60': 0}
df_reduced['X7']=df_reduced['X7'].map(d2)

In [ ]:
d1 = {'A': 1, 'B': 2,'C':3, 'D': 4, 'E': 5,'F':6,'G':7}
df_reduced['X8']=df_reduced['X8'].map(d1)

In [ ]:
df_reduced['X11'] = df_reduced['X11'].str.split(expand=True)[0]
df_reduced.replace({'X11': '<'}, 0, inplace=True)
df_reduced.replace({'X11': '10+'}, 10, inplace=True)

In [ ]:
df_reduced['X11'].value_counts()

In [ ]:
df_reduced['X11'].unique()

In [ ]:
df_reduced['X11'].isnull().sum()

In [ ]:
d = {'VERIFIED - income': 1, 'VERIFIED - income source': 1,'not verified':0}
df_reduced['X14']=df_reduced['X14'].map(d)

In [ ]:
print(len(df_reduced['X17'].unique()))
print(df_reduced['X17'].isnull().sum())
temp = df_reduced['X17'].value_counts().to_dict()
for key, num in zip(temp,range(1,len(temp)+1)):
    temp[key] = num
temp

In [ ]:
df_reduced['X17']=df_reduced['X17'].map(temp)

In [ ]:
df_reduced['X30']=df_reduced['X30'].replace({'%':''}, regex = True)
df_reduced['X30']=df_reduced['X30'].astype(float)

In [ ]:
df_reduced['X30']=df_reduced['X30'].mask(df_reduced['X30']<=30.,1)
df_reduced['X30']=df_reduced['X30'].mask(df_reduced['X30']>30,2)
df_reduced['X30']=df_reduced['X30'].mask(df_reduced['X30']>50,3)

In [ ]:
d2 = {'w': 1, 'f': 0}
df_reduced['X32']=df_reduced['X32'].map(d2)

In [ ]:
df_reduced.dtypes

In [ ]:
#Outliner Treatment
sns.distplot(df_reduced['X13'])
plt.show()
df_reduced['X13'].plot.box(figsize=(16,5))
plt.show()

In [ ]:
print(df_reduced.shape)
df_reduced.isnull().sum()

In [ ]:
# fill using the mean of the feature.
df_reduced['X30'].fillna(df_reduced['X30'].mean(),inplace=True)

In [ ]:
dt1=df_reduced['X11'].dropna()
dt1=dt1.astype(float)
dt1.mean()

In [ ]:
# fill using the mean of the feature.
df_reduced['X11'].fillna(dt1.mean(),inplace=True)
df_reduced['X11']=df_reduced['X11'].astype(float)

In [ ]:
#Applied a more conservative approach to drop the rest of nulls due to the total numbers of null from the total population. 
df_reduced.dropna(inplace=True)

In [ ]:
df_reduced.columns

In [ ]:
f,ax = plt.subplots(figsize=(12,10))
corr=df_reduced.corr()
sns.heatmap(corr,annot=True,fmt='.2f')

In [ ]:
df_reduced=df_reduced.drop(['X5'],axis=1)

In [ ]:
df_reduced

Data Cleaning/Prepping - Test Data

In [ ]:
df_test = pd.read_csv('Loan_Prediction/Holdout for Testing.csv')
df_test.head()

In [ ]:
df_test=df_test[['X4','X7','X8','X11',
        'X13', 'X14','X17', 'X21',
       'X22','X24', 'X27', 'X28', 'X29', 'X30', 'X31',
       'X32']]
print(df_test.head(5))
print(df_test.shape)
df_test.isnull().sum()

In [ ]:
df_test['X4']=df_test['X4'].replace({'\$':''}, regex = True)
df_test['X4']=df_test['X4'].replace({',':''}, regex = True)
df_test['X4']=df_test['X4'].astype(float)

In [ ]:
df_test['X7']=df_test['X7'].str.slice(1,3)

In [ ]:
d2 = {'36': 1, '60': 0}
df_test['X7']=df_test['X7'].map(d2)

In [ ]:
d1 = {'A': 1, 'B': 2,'C':3, 'D': 4, 'E': 5,'F':6,'G':7}
df_test['X8']=df_test['X8'].map(d1)

In [ ]:
df_test['X11'] = df_test['X11'].str.split(expand=True)[0]
df_test.replace({'X11': '<'}, 0, inplace=True)
df_test.replace({'X11': '10+'}, 10, inplace=True)

In [ ]:
d = {'VERIFIED - income': 1, 'VERIFIED - income source': 1,'not verified':0}
df_test['X14']=df_test['X14'].map(d)

In [ ]:
df_test['X17']=df_test['X17'].map(temp)

In [ ]:
df_test['X30']=df_test['X30'].replace({'%':''}, regex = True)
df_test['X30']=df_test['X30'].astype(float)

In [ ]:
df_test['X30']=df_test['X30'].mask(df_test['X30']<=30.,1)
df_test['X30']=df_test['X30'].mask(df_test['X30']>30,2)
df_test['X30']=df_test['X30'].mask(df_test['X30']>50,3)

In [ ]:
d2 = {'w': 1, 'f': 0}
df_test['X32']=df_test['X32'].map(d2)

In [ ]:
df_test.isnull().sum()

In [ ]:
#Outliner Treatment for X11 and X13
sns.distplot(df_test['X13'])
plt.show()
df_test['X13'].plot.box(figsize=(16,5))
plt.show()

In [ ]:
df_test['X13_New']=np.log(df_test['X13'])
df_test.drop('X13',1,inplace=True)

In [ ]:
df_test['X30'].fillna(df_test['X30'].mean(),inplace=True)

In [ ]:
print(df_test['X11'].value_counts())

In [ ]:
dt=df_test['X11'].dropna()
dt=dt.astype(float)
print(dt.mode())
dt.mean()

In [ ]:
df_test['X11'].fillna(dt.mean(),inplace=True)

In [ ]:
df_test.dtypes

In [ ]:
cols = ['X4', 'X7', 'X8', 'X11', 'X14', 'X17', 'X21', 'X22', 'X24',
       'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X13_New']
df_test[cols] = df_test[cols].applymap(np.float64)

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.shape

Modeling

Fitting data into the following models:

1. Linear Regression
2. Lasso
3. Ridge
4. Decision Tree
5. Pipelining
6. Random Forest

In [1]:
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import statsmodels.api as sm 
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import random
import warnings
warnings.simplefilter(action='ignore')

%matplotlib inline

In [ ]:
X = df_reduced.drop('X1',1)
y = df_reduced['X1']
#create x and y variables

In [ ]:
p = PolynomialFeatures()
features_poly = p.fit_transform(X)
poly_df = pd.DataFrame(features_poly, columns=p.get_feature_names())
#creating a data frame of the fit/transformed features

In [ ]:
features_poly_test = p.transform(df_test)
poly_df_test = pd.DataFrame(features_poly_test, columns=p.get_feature_names())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(poly_df, y, random_state=28)

In [ ]:
ss = StandardScaler()
ss.fit(X_train)
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [ ]:
scaled_test_data = ss.transform(poly_df_test)
#scaling the test data and setting it equal to a variable, scaled_test_data

In [ ]:
y_train_log = np.log(y_train)
y_test_log = np.log(y_test)
print(X_train.shape)
X_test.shape

Linear Regression

In [ ]:
lr = LinearRegression()

In [ ]:
print(lr.fit(X_train_sc, y_train_log))
#fitting my training data to the linear regression model
print(lr.score(X_train_sc, y_train_log))
#scoring my training data
lr.score(X_test_sc, y_test_log)
#scoring my testing data r squared

Ridge

In [ ]:
ridge = RidgeCV()

In [ ]:
print(ridge.fit(X_train_sc, y_train_log))
#fitting my training data to the ridge model
print(ridge.score(X_train_sc, y_train_log))
#scoring my training data
ridge.score(X_test_sc, y_test_log)
#scoring my testing data r squared

Lasso

In [ ]:
lasso = LassoCV()

In [ ]:
print(lasso.fit(X_train_sc, y_train_log))
#fitting my training data to the lasso model
print(lasso.score(X_train_sc, y_train_log))
#scoring my training data
lasso.score(X_test_sc, y_test_log)
#scoring my testing data r squared

In [ ]:
y_pred = lasso.predict(scaled_test_data)
#creating a variable, y_pred, and setting it equal to the lasso prediction of my scaled test data

In [ ]:
y_pred_1 = np.exp(y_pred)
y_pred_2 = np.exp(y_pred_1)

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
sns.distplot(np.exp(df_reduced['X1']), kde=False, color='#A15EDB', bins=50, label='y')
sns.distplot(y_pred_2, kde=False, color='#69547C', bins=50, label='yhat')

plt.xlabel('loan rate', fontsize=19, labelpad=11)
plt.xticks(fontsize=14)
plt.ylabel('Count', fontsize=19, labelpad=11)
plt.yticks(fontsize=14)
plt.legend(loc='upper right')

#comparing the distribution of my target variable versus my predictions

In [ ]:
pred_df = pd.DataFrame()
pred_df['Loan Rate'] = y_pred_1
pred_df.to_csv('final_submission.csv')

Decision Tree and Cross Validation Check

In [ ]:
# Import DecisionTreeRegressor from sklearn.tree
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

import sklearn.model_selection as cv
from sklearn.tree import DecisionTreeRegressor
(X_train1, X_test1, y_train1, y_test1) = cv.train_test_split(X, y, test_size=.30)

In [ ]:
ss = StandardScaler()
ss.fit(X_train1)
X_train_sc1 = ss.transform(X_train1)
X_test_sc1 = ss.transform(X_test1)

In [ ]:
# Instantiate dt
dt = DecisionTreeRegressor(max_depth=8,
                           min_samples_leaf=0.13,
                           random_state=3)

# Fit to the training set
dt.fit(X_train_sc1, y_train1)

In [ ]:
from sklearn.metrics import mean_squared_error as MSE

# Compute y_pred
y_pred = dt.predict(X_test_sc1)

# Compute mse_dt
mse_dt = MSE(y_test1, y_pred)

# Compute rmse_dt
rmse_dt = mse_dt**(1/2)

# Print rmse_dt
print("Test set RMSE of dt: {:.2f}".format(rmse_dt))

In [ ]:
from sklearn.model_selection import cross_val_score
# Compute the array containing the 10-folds CV MSEs
MSE_CV_scores = - cross_val_score(dt, X_train_sc1, y_train1, cv=10, 
                                  scoring='neg_mean_squared_error', 
                                  n_jobs=-1) 

# Compute the 10-folds CV RMSE
RMSE_CV = (MSE_CV_scores.mean())**(1/2)

# Print RMSE_CV
print('CV RMSE: {:.2f}'.format(RMSE_CV))

In [ ]:
# Import mean_squared_error from sklearn.metrics as MSE
from sklearn.metrics import mean_squared_error as MSE

# Fit dt to the training set
dt.fit(X_train_sc1, y_train1)

# Predict the labels of the training set
y_pred_train = dt.predict(X_train_sc1)

# Evaluate the training set RMSE of dt
RMSE_train = (MSE(y_train1, y_pred_train))**(1/2)

# Print RMSE_train
print('Train RMSE: {:.2f}'.format(RMSE_train))

In [ ]:
print(dt.score(X_train_sc1, y_train1))
print(dt.score(X_test_sc1, y_test1))

In [ ]:
ss_test = ss.transform(df_test)
y_pred = dt.predict(ss_test)

In [ ]:
pred_df = pd.DataFrame()
pred_df['Loan Rate'] = y_pred
pred_df.to_csv('final_submission2.csv')

Pipelining -Combining GridSearchCV and Ridge Regression Model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

In [ ]:
param={"step2__alpha":[0.01,0.1,1,10,100]}

In [ ]:
from sklearn.model_selection import GridSearchCV
pipe2=Pipeline([('step1',StandardScaler()),('step2',Ridge())])
grid=GridSearchCV(pipe2,param_grid=param,cv=5)
grid.fit(X_train,y_train)
print("Best cv accuracy : ",grid.best_score_)
print("Best parameter : ",grid.best_params_)
print("Train score : ",grid.score(X_train,y_train))
print("Test score : ",grid.score(X_test,y_test))

Random Forest Regressor

In [ ]:
# Basic imports
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)
import statsmodels.api as sm
from scipy import stats

In [ ]:
(X_train1, X_test1, y_train1, y_test1) = cv.train_test_split(X, y, test_size=.20)

In [ ]:
ss = StandardScaler()
ss.fit(X_train1)
X_train_sc1 = ss.transform(X_train1)
X_test_sc1 = ss.transform(X_test1)

In [ ]:
# Instantiate a random forests regressor 'rf'400 estimators
regrRM = RandomForestRegressor(n_estimators=400, max_depth = 50, min_samples_split = 5,min_samples_leaf =2)
regrRM.fit(X_train_sc1, y_train1)

In [ ]:
print(regrRM.score(X_train_sc1, y_train1))
print(regrRM.score(X_test_sc1, y_test1))
y_pred1= regrRM.predict(X_test_sc1)
print(np.sqrt(metrics.mean_squared_error(y_test1,y_pred1)))

In [ ]:
regrRM.feature_importances_

In [ ]:
CoeffRM1 = pd.DataFrame(columns=["Variable","FeatureImportance"])
CoeffRM1["Variable"]=X_train1.columns
CoeffRM1["FeatureImportance"]=regrRM.feature_importances_
CoeffRM1.sort_values("FeatureImportance", ascending = False)

In [ ]:
# Predict the test set labels 'y_pred'
y_pred_train=regrRM.predict(X_train_sc1)
# Evaluate the test set RMSE
rmse_test = MSE(y_test1, y_pred1)**(1/2)
rmse_train = MSE(y_train1, y_pred_train)**(1/2)
# Print the test set RMSE
print('Test set RMSE of rf: {:.2f}'.format(rmse_test))
print('Train set RMSE of rf: {:.2f}'.format(rmse_train))

Try without Standardize

In [ ]:
(X_train2, X_test2, y_train2, y_test2) = cv.train_test_split(X, y, test_size=.20)

In [ ]:
regrRM = RandomForestRegressor(n_estimators=400, min_samples_leaf=0.1)
regrRM.fit(X_train2, y_train2)

In [ ]:
# Predict the test set labels 'y_pred'
y_pred2= regrRM.predict(X_test2)
y_pred_train2=regrRM.predict(X_train2)
# Evaluate the test set RMSE
rmse_test = MSE(y_test2, y_pred2)**(1/2)
rmse_train = MSE(y_train1, y_pred_train2)**(1/2)
# Print the test set RMSE
print('Test set RMSE of rf: {:.2f}'.format(rmse_test))
print('Train set RMSE of rf: {:.2f}'.format(rmse_train))

I ran 6 models for the loan predictions

Linear Regression: this model returned the r-squarted of 0.90

Lasso: this model also returned the r-squared of 0.90

Ridge: Ridge model returned a similar r-squared of.90 comparing to linear regression model and lasso model

Decision Tree: I performed the a comparsion between CV error, train error, and test error, and determined my current model is performing well as all three errors are very close (at 1.69). This model returned the r-squared of 0.85.

Pipelining: I combined GridSearchCV and ridge regression model in my pipelining. This model returned the r-squared of 0.92.

Random Forest Regressor: Out of all my models, the random forest regresor performed the worst due to a problem of overfitting. In order to mitigate overfitting, I set max depth to 50, the default value to 2, and min_samples_leaf to 2. But I was not able to get test RMSE and train RMSE closer. As a result, I will not use Random Forest Regrssor model to predict the loan rate.

I printed out loan prediction results using Lasso (final submission) and Decision Tree methonds ( final submission 2) and validated their similarities. I will use descision tree result as my final submission as I performed my cross validation and is confident with my result.